<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/hapt%203%20folds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn.metrics
from tensorflow.keras.utils import plot_model
import datetime


In [2]:
%load_ext tensorboard


# Some methods and data which will be used below

In [3]:
# (i).   Mapping labels to their resp. classes
# (ii).  Mapping classes to their resp. labels

label_to_class = {
    1  : 'WALKING',
    2  : 'WALKING_UPSTAIRS',
    3  : 'WALKING_DOWNSTAIRS',
    4  : 'SITTING',
    5  : 'STANDING',
    6  : 'LAYING',
    7  : 'STAND_TO_SIT',
    8  : 'SIT_TO_STAND',
    9  : 'SIT_TO_LIE',
    10 : 'LIE_TO_SIT',
    11 : 'STAND_TO_LIE',
    12 : 'LIE_TO_STAND',
    np.nan : np.nan
}
class_to_label = {
    'WALKING' : 1,
    'WALKING_UPSTAIRS' : 2,
    'WALKING_DOWNSTAIRS' : 3,
    'SITTING' : 4,
    'STANDING' : 5,
    'LAYING' : 6,
    'STAND_TO_SIT' : 7,
    'SIT_TO_STAND' : 8,
    'SIT_TO_LIE' : 9,
    'LIE_TO_SIT' : 10,
    'STAND_TO_LIE' : 11,
    'LIE_TO_STAND' : 12,
    np.nan : np.nan
}

In [4]:
# (i).   Removing data-points where y and x values is null

# Other methods can be
# ffill (forward fill) => fills using forward points
# bfill (backward fill) => using backward points
# interpolate

def remove_null(xdata,ydata):
    xdata = xdata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]
    xdata = xdata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]

    return xdata,ydata

In [5]:
# normalize xdata using sklearn.preprocessing.StandardScaler and returns
# scaler object to use it furthur for testing data

# Each axis of each sensor has different min, max, I scaled according to them seperately
# Initial shape == (None,128,2,3)
# changed to (None , 6) :-
# reshape to (None,128,6) -> swapaxis(0,2) -> reshape(6,-1) -> transpose
# Fit scaler OR transform according to scaler

# Reverse above process to get back oiginal data
# transpose -> reshape(6,128,None) -> swapaxes(0,2) -> reshape(None,128,2,3)

def get_scaler(xdata):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    scaler = MinMaxScaler(feature_range = (-1,1))
    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    scaler.fit(xdata)
    return scaler

def scale_data(xdata,scaler):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    xdata = scaler.transform(xdata)
    xdata = xdata.T.reshape(sensor*axis,timestamp,row)
    xdata = np.swapaxes(xdata,0,2).reshape(row,timestamp,sensor,axis)

    return xdata

In [6]:
# takes in location, exp no., user no., start and end(end point is excluded from reading i.e lastpoint+1) point
# ,overlap array, and returns xdata and ydata

def create_windows(location, exp, user, start, end, activity, length, overlap):

    acc_file  = location + '/acc_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'
    gyro_file = location + '/gyro_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'

    acc_data  = np.loadtxt(acc_file)
    gyro_data = np.loadtxt(gyro_file)

    xtrain = []
    ytrain = []

    while (start + length <= end) :

        stop = start + length
        window = []

        while start != stop :
            window.append( [acc_data[start] , gyro_data[start]] )
            start += 1

        xtrain.append(window)
        ytrain.append(activity)

        start = stop - overlap[activity-1]

    return xtrain,ytrain

In [7]:
# location == location of file
# lenght == lenght of window
# overlap == array of overlaps of size == number of unique activities
# overlap depends on activity so as to extract more data from a particular class if needed


# (i).   Loading labels.txt as labels
# (ii).  Iterating in labels and calling create_windows on acceleration file, extending returned data in xtrain, ytrain
# (iii). Iterating in labels and calling create_windows on gyroscope file, extending returned data in xtrain, ytrain

def prepare_data(location,length = 128,overlap = [64]*12):

    xdata = []
    ydata = []

    labels = np.loadtxt(location+'/labels.txt',dtype = 'uint32')

    for exp,user,activity,start,end in labels :

        xtemp , ytemp = create_windows(location, exp, user, start, end+1, activity, length, overlap)
        xdata.extend(xtemp)
        ydata.extend(ytemp)

    return np.array(xdata),np.array(ydata)

In [8]:
# (i). Finds max element index sets its 1 and sets remaining 0
#      for each row

def to_categorical(ydata):

    for i in range(len(ydata)):
        j = ydata[i].argmax()
        for k in range(len(ydata[i])):
            ydata[i][k] = (k == j)
    return ydata

In [27]:
# (i).  OneHotEncoding ydata
# (ii). Converting sparsh matrix ydata into dense form and then matrix into numpy array

def one_hot_encoded(ydata):
    ydata = OneHotEncoder().fit_transform(ydata.reshape(len(ydata),1))
    ydata = np.asarray(ydata.todense())
    return ydata

def splitData(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  #xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)
  labels = ytrain


  ytrain = one_hot_encoded(ytrain)
  return xtrain,ytrain,labels

def splitData2(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  #xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)

  return xtrain,ytrain


def resampleData(x,y):
  x = x.reshape(5773,768)
  df = pd.DataFrame(x)

  from imblearn.over_sampling import RandomOverSampler
  from collections import Counter
  from sklearn.datasets import make_classification
  from imblearn.over_sampling import SMOTE

  oversample = SMOTE()
  df, y = oversample.fit_resample(df, y)
  df=df.to_numpy()
  x=df.reshape(12228,128,6)
  x=df.reshape(12228,128,2,3)
  return x,y

# Data Preparation

## Loading data from files

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Preparing data, xtrain, ytrain
# Last six classes [7 to 12] has very less weightage in data since they are extra classes added
# , made from original six classes
# so, I took more overlapping in them to get slightly more data

xtrain,ytrain = prepare_data('/content/drive/MyDrive/HAPT Data Set/RawData',128,[0,0,0,0,0,0,0,0,0,0,0,0])

In [12]:
originalx = xtrain
originaly=ytrain

In [ ]:
#xtrain=originalx
#ytrain = originaly

In [13]:
ytrain.shape

(5773,)

In [14]:
xtrain.shape

(5773, 128, 2, 3)

In [ ]:
128*6

768

In [15]:
xtrain = xtrain.reshape(5773,768)


In [16]:
xtrain.shape

(5773, 768)

In [17]:
df = pd.DataFrame(xtrain)
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
756,1.031945,-0.079167,0.059722,0.015577,-0.008552,0.01741,1.027778,-0.077778,0.055556,0.010385,...,0.063889,-0.007941,0.009163,0.004276,1.025,-0.081944,0.051389,0.002138,0.005498,-0.004276


In [18]:
ytrain.shape

(5773,)

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/

In [19]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
df, ytrain = oversample.fit_resample(df, ytrain)

In [20]:
xtrain.shape

(5773, 768)

In [21]:
df.shape

(12228, 768)

In [22]:
ytrain.shape

(12228,)

In [23]:
df=df.to_numpy()
xtrain=df.reshape(12228,128,6)

In [24]:
xtrain.shape

(12228, 128, 6)

In [25]:
xtrain=df.reshape(12228,128,2,3)

In [26]:
xtrain.shape

(12228, 128, 2, 3)

**start SVM**

In [28]:
X_train = pd.read_csv('/content/drive/MyDrive/newdata48features.csv', header=None)
xxx,Y_train = resampleData(originalx,originaly)

In [29]:
X_train.head() # mean ,median , std , variance , root mean square , max, min

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0__median,0__mean,0__standard_deviation,0__variance,0__root_mean_square,0__maximum,0__absolute_maximum,0__minimum,1__median,1__mean,...,4__absolute_maximum,4__minimum,5__median,5__mean,5__standard_deviation,5__variance,5__root_mean_square,5__maximum,5__absolute_maximum,5__minimum
1,1.01944450285877,1.0193142942447153,0.0024762469750240646,6.131799081315831e-06,1.01931730204617,1.027777854158295,1.027777854158295,1.0125000434425,-0.1250000020616516,-0.12428385819818219,...,0.02138028293848038,-0.02138028293848038,0.003054326167330146,0.0024673228924712025,0.006133609869028267,3.762117002544095e-05,0.006611267070777992,0.01679879426956177,0.01679879426956177,-0.01404990069568157
2,1.01944450285877,1.0199002330079632,0.0033669412028527372,1.1336293063467436e-05,1.0199057905428133,1.02916674604155,1.02916674604155,1.009722259675992,-0.1263888939449057,-0.12505425802505304,...,0.03451388701796532,-0.03451388701796532,0.00366519158706069,0.00544290474636,0.009781032110216805,9.566858914109219e-05,0.011193471365891834,0.03634648025035858,0.03634648025035858,-0.01435533352196217
3,1.01944450285877,1.0201497995182354,0.002750501804412125,7.565260176074354e-06,1.0201535074278145,1.025000070391787,1.025000070391787,1.011111151559246,-0.1305555574387036,-0.13018663702089006,...,0.02626720629632473,-0.02626720629632473,0.0,0.0003436116994635086,0.009007026194748656,8.112652087288845e-05,0.009013578083807598,0.02718350291252136,0.02718350291252136,-0.02382374368607998
4,1.020833394742025,1.0207791411528349,0.0028706455352721085,8.24060578917769e-06,1.0207831775742136,1.027777854158295,1.027777854158295,1.013888935325754,-0.1263888939449057,-0.1276367229824334,...,0.03390302136540413,-0.03390302136540413,0.005345070967450738,0.006118197258047076,0.012574714847580413,0.00015812345349795927,0.013984126400541938,0.03359758853912354,0.03359758853912354,-0.02443460933864117


In [30]:
Y_train.shape

(12228,)

In [31]:
X_train=X_train.iloc[1: , :]


In [ ]:
###########################################################################################
x1train,x1test,y1train,y1test=splitData2(X_train,Y_train,'dynamic')
x2train,x2test,y2train,y2test=splitData2(X_train,Y_train,'static')
x3train,x3test,y3train,y3test=splitData2(X_train,Y_train,'transition')

###########################################################################################
x1train.shape,y1train.shape,x3train.shape,y3train.shape

((2139, 48), (2139,), (4279, 48), (4279,))

In [ ]:
X_train = np.concatenate([x1train,x2train, x3train], axis=0)
X_test = np.concatenate([x1test,x2test, x3test], axis=0)

Y_train = np.concatenate([y1train,y2train, y3train], axis=0)
Y_test = np.concatenate([y1test,y2test, y3test], axis=0)



In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size = 0.3,random_state=42)


In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((8557, 48), (3671, 48), (8557,), (3671,))

In [ ]:
X_test.shape,Y_test.shape

((3671, 48), (3671,))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

tree_list = [200]
scores=[]
for n_trees in tree_list:

    rf = RandomForestClassifier(n_estimators=n_trees, random_state=40)
    rf.fit(x2train, y2train)
    rf_pred = rf.predict(x2test)
    scores.append(accuracy_score(y2test, rf_pred))
    print('Accuracy score of static activities TEST: {:.3f}'.format(accuracy_score(y2test, rf_pred)*100))

Accuracy score of static activities TEST: 95.643
Accuracy score of static activities TEST: 95.534
Accuracy score of static activities TEST: 95.643
Accuracy score of static activities TEST: 95.752


**end** **svm**

## Handling missing data and Normalizing data

In [32]:
xtrain,ytrain = remove_null(xtrain,ytrain)

In [33]:
xtrain.shape

(12228, 128, 2, 3)

In [ ]:
2449+2960+364

5773

In [34]:
ytrain.shape

(12228,)

**Hier split and training**

**Dynamic**

In [36]:

x11train,y11train=splitData2(X_train,Y_train,'dynamic')
x22train,y22train=splitData2(X_train,Y_train,'static')
x33train,y33train=splitData2(X_train,Y_train,'transition')

x11train=x11train.reset_index(level=None, drop=True)
x22train=x22train.reset_index(level=None, drop=True)
x33train=x33train.reset_index(level=None, drop=True)

dynamicxtrain1,dynamicytrain1,dynamiclabels1 = splitData(xtrain,ytrain,'dynamic')
staticxtrain1,staticytrain1,staticlabels1 = splitData(xtrain,ytrain,'static')
transitionxtrain1,transitionytrain1,transitionlabels1 = splitData(xtrain,ytrain,'transition')



In [37]:
dynamicxtrain1.shape,x11train.shape

((3057, 128, 2, 3), (3057, 48))

In [38]:
folds = 3

In [57]:
import numpy as np
from sklearn.model_selection import KFold
dynamic_fold_train=[]
dynamic_fold_test=[]
kf = KFold(n_splits=folds)
kf.get_n_splits(dynamicxtrain1)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(dynamicxtrain1)):
    dynamic_fold_train.append(train_index)
    dynamic_fold_test.append(test_index)

static_fold_train=[]
static_fold_test=[]
kf = KFold(n_splits=folds)
kf.get_n_splits(staticxtrain1)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(staticxtrain1)):
    static_fold_train.append(train_index)
    static_fold_test.append(test_index)

transition_fold_train=[]
transition_fold_test=[]
kf = KFold(n_splits=folds)
kf.get_n_splits(transitionxtrain1)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(transitionxtrain1)):
    transition_fold_train.append(train_index)
    transition_fold_test.append(test_index)


KFold(n_splits=3, random_state=None, shuffle=False)
KFold(n_splits=3, random_state=None, shuffle=False)
KFold(n_splits=3, random_state=None, shuffle=False)


In [58]:
import numpy as np
from sklearn.model_selection import KFold
dynamic_fold_train2=[]
dynamic_fold_test2=[]
kf = KFold(n_splits=folds)
kf.get_n_splits(x11train)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(x11train)):
    dynamic_fold_train2.append(train_index)
    dynamic_fold_test2.append(test_index)
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

static_fold_train2=[]
static_fold_test2=[]
kf = KFold(n_splits=folds)
kf.get_n_splits(x22train)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(x22train)):
    static_fold_train2.append(train_index)
    static_fold_test2.append(test_index)

transition_fold_train2=[]
transition_fold_test2=[]
kf = KFold(n_splits=folds)
kf.get_n_splits(x33train)
print(kf)
for i, (train_index, test_index) in enumerate(kf.split(x33train)):
    transition_fold_train2.append(train_index)
    transition_fold_test2.append(test_index)

KFold(n_splits=3, random_state=None, shuffle=False)
Fold 0:
  Train: index=[1019 1020 1021 ... 3054 3055 3056]
  Test:  index=[   0    1    2 ... 1016 1017 1018]
Fold 1:
  Train: index=[   0    1    2 ... 3054 3055 3056]
  Test:  index=[1019 1020 1021 ... 2035 2036 2037]
Fold 2:
  Train: index=[   0    1    2 ... 2035 2036 2037]
  Test:  index=[2038 2039 2040 ... 3054 3055 3056]
KFold(n_splits=3, random_state=None, shuffle=False)
KFold(n_splits=3, random_state=None, shuffle=False)


In [59]:
from sklearn.ensemble import RandomForestClassifier
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

accs=[]
for fold in range(folds):


  #dynamicxtrain,dynamicxtest,dynamicytrain,dynamicytest,dynamiclabels
  dynamicxtest = dynamicxtrain1[dynamic_fold_test[fold]]
  dynamicytest = dynamicytrain1[dynamic_fold_test[fold]]
  dynamicytrain = dynamicytrain1[dynamic_fold_train[fold]]
  dynamicxtrain = dynamicxtrain1[dynamic_fold_train[fold]]
  dynamiclabels = dynamiclabels1[dynamic_fold_test[fold]]

  #staticxtrain,staticxtest,staticytrain,staticytest,staticlabels

  staticxtest = staticxtrain1[static_fold_test[fold]]
  staticytest = staticytrain1[static_fold_test[fold]]
  staticytrain = staticytrain1[static_fold_train[fold]]
  staticxtrain = staticxtrain1[static_fold_train[fold]]
  staticlabels = staticlabels1[static_fold_test[fold]]



  transitionxtest = transitionxtrain1[transition_fold_test[fold]]
  transitionytest = transitionytrain1[transition_fold_test[fold]]
  transitionytrain = transitionytrain1[transition_fold_train[fold]]
  transitionxtrain = transitionxtrain1[transition_fold_train[fold]]
  transitionlabels = transitionlabels1[transition_fold_test[fold]]
##################x2train,y2train##################################################
  dynamicxtrain=dynamicxtrain.reshape(dynamicxtrain.shape[0],dynamicxtrain.shape[1],6)
  dynamicxtest=dynamicxtest.reshape(dynamicxtest.shape[0],dynamicxtest.shape[1],6)

  staticxtrain=staticxtrain.reshape(staticxtrain.shape[0],staticxtrain.shape[1],6)
  staticxtest=staticxtest.reshape(staticxtest.shape[0],staticxtest.shape[1],6)

  transitionxtrain=transitionxtrain.reshape(transitionxtrain.shape[0],transitionxtrain.shape[1],6)
  transitionxtest=transitionxtest.reshape(transitionxtest.shape[0],transitionxtest.shape[1],6)



  x1test = x11train.iloc[dynamic_fold_test2[fold]]
  y1test = y11train[dynamic_fold_test2[fold]]
  y1train = y11train[dynamic_fold_train2[fold]]
  x1train = x11train.iloc[dynamic_fold_train2[fold]]

  x2test = x22train.iloc[static_fold_test2[fold]]
  y2test = y22train[static_fold_test2[fold]]
  y2train = y22train[static_fold_train2[fold]]
  x2train = x22train.iloc[static_fold_train2[fold]]

  x3test = x33train.iloc[transition_fold_test2[fold]]
  y3test = y33train[transition_fold_test2[fold]]
  y3train = y33train[transition_fold_train2[fold]]
  x3train = x33train.iloc[transition_fold_train2[fold]]


  from sklearn.ensemble import RandomForestClassifier
  from sklearn.metrics import confusion_matrix, accuracy_score

  tree_list = [100]
  scores=[]
  for n_trees in tree_list:

    rf = RandomForestClassifier(n_estimators=n_trees, random_state=40)
    rf.fit(x2train, y2train)
    rf_pred = rf.predict(x2test)
    scores.append(accuracy_score(y2test, rf_pred))
    print('Accuracy score TEST: {:.3f}'.format(accuracy_score(y2test, rf_pred)*100))

  timesteps = len(dynamicxtrain[0])
  input_dim = len(dynamicxtrain[0][0])
  n_classes =3

  print(timesteps)
  print(input_dim)
  print(len(X_train))

  epochs = 100
  batch_size = 16
  n_hidden = 64
  model1 = Sequential()
  # Configuring the parameters
  model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
  # Adding a dropout layer
  model1.add(Dropout(0.5))
  model1.add(LSTM(32))
  model1.add(Dropout(0.5))
  model1.add(Dense(n_classes, activation='softmax'))
  model1.summary()

  model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
  model1.fit(dynamicxtrain,
          dynamicytrain,
          batch_size=batch_size,
          #validation_data=(dynamicxtest, dynamicytest),
          epochs=epochs,
          )


  timesteps = len(transitionxtrain[0])
  input_dim = len(transitionxtrain[0][0])
  n_classes =6

  print(timesteps)
  print(input_dim)
  print(len(X_train))
  model2 = Sequential()
  # Configuring the parameters
  model2.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
  # Adding a dropout layer
  model2.add(Dropout(0.5))
  model2.add(LSTM(32))
  model2.add(Dropout(0.5))
  model2.add(Dense(n_classes, activation='softmax'))
  model2.summary()

  model2.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
  model2.fit(transitionxtrain,
          transitionytrain,
          batch_size=batch_size,
          #validation_data=(dynamicxtest, dynamicytest),
          epochs=epochs,
          )

  X_train = np.concatenate([x1train,x2train,x3train], axis=0)
  X_test = np.concatenate([x1test,x2test,x3test], axis=0)

  yexpect = np.concatenate([dynamiclabels,staticlabels,transitionlabels], axis=0)
  merge_arr = np.concatenate([dynamicxtest, staticxtest,transitionxtest], axis=0)
  scores=[]
  dynamiccount=0
  staticcount=0
  transitioncount=0
  dynamics = np.array([1,2,3])
  statics = np.array([4,5,6])
  transitions = np.array([7,8,9,10,11,12])
  for i in range(len(merge_arr)):#len(xtest)
      row = merge_arr[i].reshape(1,128,6)
      if(yexpect[i] in [1,2,3]):
          predict_x=model1.predict(row)

          pred=dynamics[np.argmax(predict_x,axis=1)]
          scores.append(pred[0])

      elif(yexpect[i] in [4,5,6]):
        s = rf.predict(X_test[i].reshape(1, -1))  #x2test.loc[0,:].to_numpy().reshape(1,-1)
        scores.append(s[0])

      else:
        predict_x=model2.predict(row)

        pred=transitions[np.argmax(predict_x,axis=1)]
        scores.append(pred[0])


      print(i)
  print('dynamic count '+str(dynamiccount)+' static count '+str(staticcount)+' transition count '+str(transitioncount))
  from sklearn.metrics import confusion_matrix, accuracy_score


  print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))
  accs.append(accuracy_score(yexpect, scores)*100)

Streaming output truncated to the last 5000 lines.
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305

In [61]:
####################################################################################################################################
accs,mean(accs)

([70.53483807654564, 95.38763493621197, 94.43081452404319], 86.78442917893359)